In [ ]:
import wandb
import torch,torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch.nn import *
from torch.optim import *
from tqdm import tqdm
device = 'cuda'
PROJECT_NAME = 'Tesla-Stock-Prediction'

In [ ]:
data = pd.read_csv('./data.csv')

In [ ]:
data.head()

In [ ]:
data = data['High']

In [ ]:
data = torch.from_numpy(np.array(data.tolist()))

In [ ]:
class Model(Module):
    def __init__(self,neurons=64):
        super().__init__()
        self.neurons = neurons
        self.lstm1 = LSTMCell(1,self.neurons).to(device)
        self.lstm2 = LSTMCell(self.neurons,self.neurons).to(device)
        self.lstm3 = LSTMCell(self.neurons,self.neurons).to(device)
        self.linear = Linear(self.neurons,1).to(device)
    
    def forward(self,X,future=0):
        outputs = []
        samples = X.shape[0]
        h_t1 = torch.zeros(samples,self.neurons,dtype=torch.float32)
        c_t1 = torch.zeros(samples,self.neurons,dtype=torch.float32)
        h_t2 = torch.zeros(samples,self.neurons,dtype=torch.float32)
        c_t2 = torch.zeros(samples,self.neurons,dtype=torch.float32)
        h_t3 = torch.zeros(samples,self.neurons,dtype=torch.float32)
        c_t3 = torch.zeros(samples,self.neurons,dtype=torch.float32)
        for X_batch in X.split(1,dim=1):
            h_t1 = h_t1.to(device)
            c_t1 = c_t1.to(device)
            h_t2 = h_t2.to(device)
            c_t2 = c_t2.to(device)
            h_t3 = h_t3.to(device)
            c_t3 = c_t3.to(device)
            X_batch = X_batch.to(device)
            h_t1,c_t1 = self.lstm1(X_batch,(h_t1,c_t1))
            h_t1 = h_t1.to(device)
            c_t1 = c_t1.to(device)
            h_t2,c_t2 = self.lstm2(h_t1,(h_t2,c_t2))
            h_t2 = h_t2.to(device)
            c_t2 = c_t2.to(device)
            h_t3,c_t3 = self.lstm3(h_t2,(h_t3,c_t3))
            h_t3 = h_t3.to(device)
            c_t3 = c_t3.to(device)
            preds = self.linear(h_t3)
            outputs.append(preds)
        for i in range(future):
            h_t1,c_t1 = self.lstm1(X_batch,(h_t1,c_t1))
            h_t2,c_t2 = self.lstm1(h_t1,(h_t2,c_t2))
            h_t3,c_t3 = self.lstm1(h_t2,(h_t3,c_t3))
            preds = self.linear(h_t3)
            outputs.append(preds)
        outputs = torch.cat(outputs,dim=1)
        return outputs

In [ ]:
data_input = data.view(1,-1)[:1,:-1].to(device).float()
data_target = data.view(1,-1)[:1,1:].to(device).float()

In [ ]:
model = Model().to(device)
criterion = MSELoss()
optimizer = LBFGS(model.parameters(),lr=0.8)

In [ ]:
epochs = 100

In [ ]:
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in range(epochs):
    def closure():
        optimizer.zero_grad()
        preds = model(data_input.float())
        loss = criterion(preds,data_target)
        wandb.log({'loss':loss.item()})
        loss.backward()
    optimizer.step(closure)
    with torch.no_grad():
        preds = model(data_input,100)
        loss = criterion(preds[:,:-100],data_target)
        wandb.log({'loss val':loss.item()})
    plt.figure(figsize=(12,6))
    plt.title(f'Step : {i+1}')
    n = train_input.shape[1]
    def draw(y_i,color):
        plt.plot(np.arange(n),y_i[:n],color)
        plt.plot(np.arange(n,n+future),y_i[n:],color + ':')
    draw(preds[0],'r')
    plt.savefig('./preds/predict%d.png'%i)
    plt.close()